# 로지스틱 회귀

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/4-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

---
# <span style="color:gold">신입생(1학년) 데이터를 활용한 휴학/자퇴/제적 예측 머신러닝  </span>

version 0.9 (2023.07.26)

***

### 사용데이터
- 2000년~2022년 기간동안의 1학년 1학기 데이터 (4207명) 
- 휴학. 1학년 여름방학 ~ 1학년 겨울방학 이전까지 **휴학**한 신입생(1학년) 학생수 (2000명)
- 자퇴. 1학년 여름방학 ~ 1학년 겨울방학 이전까지 **자퇴**한 신입생(1학년) 학생수 (946명)
- 제적. 1학년 여름방학 ~ 1학년 겨울방학 이전까지 **제적**한 신입생(1학년) 학생수 (291명)
* 데이터의 추출은 해당 기간내에 자퇴와 제적학생들을 거의 전수로 추출한 후 전체 인원이 4000명이 되도록 휴학과 자퇴인원을 최근 데이터부터 임의로 선정함

### 정리데이터
- 카테고리데이터 : 학번/검정고시여부/고교유형구분코드/고교위치/모집구분코드/전형구분코드
- 정량데이터 : 중간고사/기말고사/출석점수/평점평균/백분위점수/상담건수/비교과건수/장학금
- 최종예측대상 데이터 : 학적구분(4개의 카테고리를 두개로 묶어 <span style="color.red">이진분류(휴학/자퇴/제적, 재학)</span>로 변환


### 모델설계 사용 데이터
- 모델설계를 위해 사용한 데이터는 정량데이터 8개만을 사용하여 예측모델 구현

### 적용 머신러닝 알고리즘
- 2 진분류 Logistic Regressiojn 적용
- (참고) SVG 알고리즘으로 Recussive하게도 모델링 해보았지만 성능상 큰 차이가 없어 Logistic Regression으로만 정리함




***
## [<span style="color:gold">STEP01</span>] 데이터 준비하기 
***

- data_v1.4_preprocessing_bykim_csv : (이진분류 (1. 휴학/자퇴/제적, 2. 재학))
  
  - 전체 데이터 중 선형 정량 데이터만 csv로 구성
  - 8개의 입력과 1개의 출력이 나오는 이진분류
  - target --> scolarregister : 1.자퇴/제적/휴학 2.재학
<br><br><br>
- 필드의 순서
  - midtest	(중간고사)
  - finaltest	(기말고사)
  - attendance (출석점수)
  - avggrade (평점평균)
  - percentgrade (백분위점수)
  - counselnum (상담건수)
  - extralecture (비교과건수)
  - scalarship (장학금)
  - scolarregister ==> tartget (학적상태)
  <br><br><br>
- .csv 파일은 .py와 동일한 폴더에 위치시킴

In [53]:
#1 dhkim : csv 파일 로딩 + 처음 3개 데이터 확인

import pandas as pd
register_data = pd.read_csv('.//data_v1.4_preprocessing_bykim_csv.csv')
register_data.head()


,midtest,finaltest,attendance,avggrade,percentgrade,counselnum,extralecture,scalarship,scolarregister
0,27,38,30,4.5,100,1,4,251000,1
1,29,39,30,4.5,99,2,1,3325000,2
2,29,39,29,4.5,99,2,1,3325000,2
3,30,40,30,4.5,99,1,2,3921000,2
4,29,39,30,4.5,99,1,3,958960,2


- Target 이진 데이터 값 확인
  
  - 최종 분류기준이 되는 학적상태(scolarregister)에 저장된 종류 확인
  - 실행결과 = 이진분류라서 1(자퇴/제적/휴학), 2(재학) 두 가지 상태만 나옴

In [54]:
#2 dhkim : 분류기준에 포함된 데이터 종류 출력 (이진의 경우 1(제적,휴학,자퇴)/2(재학))

print(pd.unique(register_data['scolarregister']))


[1 2]


***
## [<span style="color:gold">STEP02</span>] 데이터 입력 및 출력 컬럼 정의하기
***

- 입력데이터 컬럼 정의
  - 8개의 입력데이터 컬럼을 register_data_input으로 정의

In [55]:
#3 dhkim : (학적상태를 예측하기 위해 사용한 8개의 입력 파라미터 (중간고사점수	기말고사점수	출석점수	평점평균	백분위점수	상담건수	비교과건수	장학금))

register_data_input = register_data[['midtest', 'finaltest', 'attendance', 'avggrade', 'percentgrade', 'counselnum', 'extralecture', 'scalarship']].to_numpy()


- 입력데이터 확인
  
  -  상위 5개의 데이터 셋 출력확인

In [56]:
#4 dhkim : (8개 입력 파라미터의 수치 데이터 중 상위 5개를 확인)

print(register_data_input[:5])


[[2.7000e+01 3.8000e+01 3.0000e+01 4.5000e+00 1.0000e+02 1.0000e+00
  4.0000e+00 2.5100e+05]
 [2.9000e+01 3.9000e+01 3.0000e+01 4.5000e+00 9.9000e+01 2.0000e+00
  1.0000e+00 3.3250e+06]
 [2.9000e+01 3.9000e+01 2.9000e+01 4.5000e+00 9.9000e+01 2.0000e+00
  1.0000e+00 3.3250e+06]
 [3.0000e+01 4.0000e+01 3.0000e+01 4.5000e+00 9.9000e+01 1.0000e+00
  2.0000e+00 3.9210e+06]
 [2.9000e+01 3.9000e+01 3.0000e+01 4.5000e+00 9.9000e+01 1.0000e+00
  3.0000e+00 9.5896e+05]]


- 출력데이터 컬럼 정의
  
  - 마지막 데이터인 scolarregister 값을 타겟 데이터로 정의

In [57]:
#5 dhkim : 타겟으로 사용할 실제 학적정보 데이터 추출

register_data_target = register_data['scolarregister'].to_numpy()


***
## [<span style="color:gold">STEP03</span>] 학습데이터와 테스트데이터 분할(split)
***

- 전체 데이터(4207개) 분할 (학습 + 테스트)

    - 학습데이터 : 3155 : 모델 설계에 사용되는 학습용 데이터
    - 테스트데이터 : 1052 : 학습이후의 모델을 이용하여 성능을 테스트하기 위해 사용되는 데이터

In [58]:
#6 dhkim : 원본 데이터를 학습 데이터 + 테스트 데이터로 분류 (overfitting 방지를 위해)

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    register_data_input, register_data_target, random_state=42)

print(train_input.shape) #3155/4207
print(test_input.shape) #1052/4207

(3155, 8)
(1052, 8)


***
## [<span style="color:gold">STEP04</span>] 입력데이터의 스케일링
***

- 입력데이터 스케일링

    - 입력데이터의 가중치 편중을 방지하기 위해 표준 스케일링 적용

In [59]:
#7 dhkim : 입력데이터를 동일한 스케이로 스케일링 함으로써 입력 파라미터 편향성 방지 (이 부분을 skip하면 테스트 예측율이 72%-->48%로 떨어짐)

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)


- 스케일링된 입력데이터 확인
  
    - 표준 스케일링 된 입력데이터의 상위 5개 확인
    - 전체적으로 동일 스케일을 가짐

In [60]:
#8 dhkim : 스케일링 된 데이터 중 상위 5개 확인

# trani_scaled = train_input
# test_scaled = test_input

print(train_scaled[:5])


[[ 0.49701052  0.14099403 -0.14262392  0.17362412  0.24016117 -0.42832932
  -0.98690041 -1.00011178]
 [-0.02777803  0.25018308  0.18848606 -0.00930595  0.13920741 -0.42832932
   0.81360313 -0.83104737]
 [ 0.49701052  0.7961283   0.18848606  0.63094928  0.54302247 -0.42832932
  -0.38673256 -1.00011178]
 [-1.07735513 -1.60603069 -1.30150882 -1.56421152 -1.17319157  1.2975698
  -0.98690041 -0.27668338]
 [ 0.84686955  0.25018308  0.68515102  0.99680942  0.79540689 -0.42832932
   0.21343529 -0.83104737]]


***
## [<span style="color:gold">STEP05</span>] 로지스틱 회귀(LogisticRegression)로 이진 분류 수행
***

- 로지스틱 회귀로 이진 분류 수행하기

    - 학습데이터(3155개)를 이용한 로지스틱회귀 모델 적용
    - 학습데이터에서의 정확도 / 테스트에서의 정확도 확인

In [61]:
#9 dhkim : LogisticRegression을 이용한 분류 수행 (이진/다중) + 예측정확도 산출

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target) #모델생성

print(lr.score(train_scaled, train_target)) # train 데이터에서의 예측률 
print(lr.score(test_scaled, test_target)) # test 데이터에서의 에측률

# 현재 2진분류의 경우 70% 정도 나옴 (실제 데이터의 수가 1:1 정도이기 때문에 임의 예측 확률은 50%임 (데이터 편향성 없음))
# v0.9에 사용된 데이터의 경우 정확한 데이터는 학습용데이터 71.69572%, 테스트용 데이터 72.62357%임


0.7169572107765452
0.7262357414448669


***
## [<span style="color:gold">STEP06</span>] 생성모델을 활용한 예측 결과 확인
***

- train 데이터 + test 데이터 최초 10개 예측결과 비교

    - train 데이터 최초 10개 비교 (9개/10개 예측 정확도)
    - test 데이터 최초 10개 비교 (6개/10개 예측 정확도)

In [62]:
#10. dhkim : 생성한 모델을 이용하여 train 데이터 처음 10개 및 test 데이터 처음 10개를 예측한 결과와 실제 데이터를 비교함

print(lr.predict(train_scaled[:10]))
print(train_target[:10])

print(lr.predict(test_scaled[:10]))
print(test_target[:10])


[1 2 2 1 1 2 1 1 2 2]
[1 2 1 1 1 2 1 1 2 2]
[2 1 2 1 1 1 2 1 2 1]
[1 2 1 1 1 1 2 2 2 1]


***
## [<span style="color:gold">STEP07</span>] 예측값을 각각 확률로 계산 (즉, 임계값 적용전)
***

In [63]:
#11. dhkim : 생성한 모델을 이용하여 (자퇴/휴학/제적) 확률과 재학확률을 %로 산출하는 코드 (입력데이터를 생성하여 scaling 한후 입력)

# print([[ 0.65048643,  0.44514596,  0.33714023,  0.41518783,  0.36659923,  0.39726523, 0.20162298, -0.83899854]])
# print(lr.predict_proba([[ 0.65048643,  0.44514596,  0.33714023,  0.41518783,  0.36659923,  0.39726523, 0.20162298, -0.83899854]]))
print(lr.predict_proba(train_scaled[:5])) # train 데이터 상위 5개에 대한 예측 확률 12211의 순으로 선택됨

# *** 이를 코드화 하여 Threshold를 0.5가 아닌 0.6, 0.7이상으로 하였을때 (제적/휴학/자퇴) 예측률을 올려 상담 대상을 추출하는 코드 작성 ***


[[0.70673136 0.29326864]
 [0.40329554 0.59670446]
 [0.46227054 0.53772946]
 [0.90187178 0.09812822]
 [0.65170707 0.34829293]]


***
## [<span style="color:gold">STEP08</span>] 예측 Taget종류와 가중치 및 바이어스 확인
***

- 타겟 클래스 + 가중치/바이어스

    - 타겟 클래스 (1: 휴학/자퇴/제적, 2: 재학)
    - 가중치 및 바이어스 
      - 1.midtest	(중간고사)
      - 2.finaltest	(기말고사)
      - 3.attendance (출석점수)
      - 4.avggrade (평점평균)
      - 5.percentgrade (백분위점수)
      - 6.counselnum (상담건수)
      - 7.extralecture (비교과건수)
      - 8.scalarship (장학금)
      - scolarregister ==> tartget (학적상태)

In [64]:
#12. dhkim : 현재 예측하고 있는 target의 종류 추출 + 로지스틱회귀모델에서의 가중치와 바이어스 (영향을 주는 정도) 값 확인

print(lr.classes_) # target 클래스 추출
print(lr.coef_, lr.intercept_) # 로지스틱회귀모델에서의 가중치와 바이어스

# 가중치의 절대값 크기순서 : 2. 기말고사(1.6095)-->7. 평점평균(-0.84212)-->7. 비교과(0.484227)-->8.장학금(0.17124)-->3.출석점수(0.142534)-->6.상담건수(0.091025)-->5.백분위점수(0.077858)-->1.중간고사(-0.00262)
# 일반적으로 생각하는 것과는 다소 다르게 나왔음


[1 2]
[[-0.00262386  1.60959221  0.1425343  -0.84212373  0.07785827  0.09102498
   0.48422687  0.17124042]] [-0.26922519]


***
## [<span style="color:gold">STEP09</span>] 모델을 이용하여 예측 확률 출력 (이진분류)
***

- 이진분류이기 때문에 양성(2:재학)의 확률만 나타남
  
  - 양성일 확률(재학(:2)일 확률)>=0.5 이면 재학 그렇지 않은 경우 휴학/자퇴/제적임
  - 이 값의 임계치(threshold)를 조정하여 중도탈락 예상자 축소필요

In [65]:
#13. dhkim : 결과를 예측하는데 사용하는 decision 함수: 2진 분류의 경우 두번째 즉 여기서는 재학률(target 2)의 확률이 나옴

from scipy.special import expit
decisions = lr.decision_function(train_scaled[:5])
#print(decisions)
print(expit(decisions)) #decision을 위해 사용한 값 (여기서는 2 즉, 재학의 확률이 나옴)


[0.29326864 0.59670446 0.53772946 0.09812822 0.34829293]


***
## [<span style="color:gold">STEP10</span>] 예측 임계값 조정을 통한 휴학/자퇴/제적 예측 확률 높이기 (매트릭, 임계값조정)
***

- 예측 확률상에서 0.5가 아닌 0.6/0.7/0.8/0.9 이상을 찾아 아래의 네 가지 경우 중 1(제적/휴학/자퇴) 2(재학)

        1 -> 1 : 실제값(휴학/자퇴/제적) - 예측값(휴학/자퇴/제적) ==> 가장 관심있는 영역
        1 -> 2 : 실제값(휴학/자퇴/제적) - 예측값(재학)
        2 -> 1 : 실제값(재학) - 예측값(휴학/자퇴/제적)
        2 -> 2 : 실제값(재학) - 예측값(재학)

        **1 -> 1 : 실제값(휴학/자퇴/제적) - 예측값(휴학/자퇴/제적)의 예측 확률을 올리는 방안 마련**

- 임계값이 = 0.5 (default) 일때의 매트릭 (Default)

    - 임계값의 default 값 0.5를 사용 (lr.predict 사용)
    - confusion_matrix (실제값, 예측값) : hit 매트릭 산출

In [77]:
from sklearn.metrics import confusion_matrix
predicted_value = lr.predict(train_scaled)
print(predicted_value)
print(train_target.shape)
confusion_matrix(train_target, predicted_value)
# print(lr.predict(train_scaled[:10]))
# print(train_target[:10])
# confusion_matrix(df.species, y_pred)
# confusion_matrix([0,1,1,0,1,0], [0,1,1,1,0,1]) #첫번째 매개변수 : 실제값, 두번째 매개변수 예측값


[1 2 2 ... 1 1 1]
(3155,)


array([[1166,  502],
       [ 391, 1096]], dtype=int64)

- 임계값에 따른 예측을 위해 예측 확률값으로 데이터 셋 구성

    - trainData 대상 : probabilityOfQuitForTrainData = 샘플수 * 2의 크기를 가짐 (0-column: 휴학/자퇴/제적 확률, 1-column: 재학확률)
    - testData 대상 : probabilityOfQuitForTestData = 샘플수 * 2의 크기를 가짐 (0-column: 휴학/자퇴/제적 확률, 1-column: 재학확률)

In [110]:
#print(lr.predict_proba(train_scaled)) # train 데이터 상위 5개에 대한 예측 확률 
probabilityOfQuitForTrainData = lr.predict_proba(train_scaled)[:,0] #제적/휴학/자퇴 할 확률 (train_scaled)
#print(lr.predict_proba(train_scaled)) # train 데이터 상위 5개에 대한 예측 확률 
probabilityOfQuitForTestData = lr.predict_proba(test_scaled)[:,0] #제적/휴학/자퇴 할 확률 (test_scaled)

# print(probabilityOfQuitForTrainData)
# print(probabilityOfQuitForTestData)

# print(probabilityOfQuitForTrainData.shape)
# print(probabilityOfQuitForTestData.shape)



- Train_scaled + Test_scaled 데이터의 예측 매트릭을 구하기 위한 데이터 행렬 선언
  
    - probabilityOfQuitForTrainData_Th_XX : 대상 데이터 = Train_scaled, 임계값 = XX
    - probabilityOfQuitForTestData_Th_XX : 대상 데이터 = Test_scaled, 임계값 = XX

    - 기본값이 1로 세팅되어 있어 2인 경우만 수정해줌

In [111]:
import numpy as np;

probabilityOfQuitForTrainData_Th_50=np.ones(probabilityOfQuitForTrainData.shape[0])
probabilityOfQuitForTrainData_Th_60=np.ones(probabilityOfQuitForTrainData.shape[0])
probabilityOfQuitForTrainData_Th_70=np.ones(probabilityOfQuitForTrainData.shape[0])
probabilityOfQuitForTrainData_Th_80=np.ones(probabilityOfQuitForTrainData.shape[0])
probabilityOfQuitForTrainData_Th_90=np.ones(probabilityOfQuitForTrainData.shape[0])

probabilityOfQuitForTestData_Th_50=np.ones(probabilityOfQuitForTestData.shape[0])
probabilityOfQuitForTestData_Th_60=np.ones(probabilityOfQuitForTestData.shape[0])
probabilityOfQuitForTestData_Th_70=np.ones(probabilityOfQuitForTestData.shape[0])
probabilityOfQuitForTestData_Th_80=np.ones(probabilityOfQuitForTestData.shape[0])
probabilityOfQuitForTestData_Th_90=np.ones(probabilityOfQuitForTestData.shape[0])


- 임계값이 = 0.5 (임계값을 custom으로 설정하는 경우) 일때의 매트릭 (Threshold 설정)

    - 임계값 Threshold = 0.5 일때의 Train_scaled 데이터 대상 / Test_sacled 데이터 대상

In [121]:
Threshold = 0.5

# train_scaled
# probabilityOfQuitForTrainData_Th_50[probabilityOfQuitForTrainData>Threshold]=1
probabilityOfQuitForTrainData_Th_50[probabilityOfQuitForTrainData<=Threshold]=2
#print(probabilityOfQuitForTrainData_Th_50)
print(confusion_matrix(train_target, probabilityOfQuitForTrainData_Th_50))

# test_scaled
# probabilityOfQuitForTestData_Th_50[probabilityOfQuitForTestData>Threshold]=1
probabilityOfQuitForTestData_Th_50[probabilityOfQuitForTestData<=Threshold]=2
#print(probabilityOfQuitForTestData_Th_50)
print(confusion_matrix(test_target, probabilityOfQuitForTestData_Th_50))


[[1166  502]
 [ 391 1096]]
[[372 167]
 [121 392]]


- 임계값이 = 0.6 (임계값을 custom으로 설정하는 경우) 일때의 매트릭 (Threshold 설정)

    - 임계값 Threshold = 0.6 일때의 Train_scaled 데이터 대상 / Test_sacled 데이터 대상

In [125]:
Threshold = 0.6

# train_scaled
# probabilityOfQuitForTrainData_Th_60[probabilityOfQuitForTrainData>Threshold]=1
probabilityOfQuitForTrainData_Th_60[probabilityOfQuitForTrainData<=Threshold]=2
#print(probabilityOfQuitForTrainData_Th_60)
print(confusion_matrix(train_target, probabilityOfQuitForTrainData_Th_60))

# test_scaled
# probabilityOfQuitForTestData_Th_60[probabilityOfQuitForTestData>Threshold]=1
probabilityOfQuitForTestData_Th_60[probabilityOfQuitForTestData<=Threshold]=2
#print(probabilityOfQuitForTestData_Th_60)
print(confusion_matrix(test_target, probabilityOfQuitForTestData_Th_60))


[[ 881  787]
 [ 136 1351]]
[[277 262]
 [ 35 478]]


- 임계값이 = 0.7 (임계값을 custom으로 설정하는 경우) 일때의 매트릭 (Threshold 설정)

    - 임계값 Threshold = 0.7 일때의 Train_scaled 데이터 대상 / Test_sacled 데이터 대상

In [126]:
Threshold = 0.7

# train_scaled
# probabilityOfQuitForTrainData_Th_70[probabilityOfQuitForTrainData>Threshold]=1
probabilityOfQuitForTrainData_Th_70[probabilityOfQuitForTrainData<=Threshold]=2
#print(probabilityOfQuitForTrainData_Th_70)
print(confusion_matrix(train_target, probabilityOfQuitForTrainData_Th_70))

# test_scaled
# probabilityOfQuitForTestData_Th_70[probabilityOfQuitForTestData>Threshold]=1
probabilityOfQuitForTestData_Th_70[probabilityOfQuitForTestData<=Threshold]=2
#print(probabilityOfQuitForTestData_Th_70)
print(confusion_matrix(test_target, probabilityOfQuitForTestData_Th_70))


[[ 588 1080]
 [  53 1434]]
[[188 351]
 [ 14 499]]


- 임계값이 = 0.8(임계값을 custom으로 설정하는 경우) 일때의 매트릭 (Threshold 설정)

    - 임계값 Threshold = 0.8 일때의 Train_scaled 데이터 대상 / Test_sacled 데이터 대상

In [127]:
Threshold = 0.8

# train_scaled
# probabilityOfQuitForTrainData_Th_80[probabilityOfQuitForTrainData>Threshold]=1
probabilityOfQuitForTrainData_Th_80[probabilityOfQuitForTrainData<=Threshold]=2
#print(probabilityOfQuitForTrainData_Th_80)
print(confusion_matrix(train_target, probabilityOfQuitForTrainData_Th_80))

# test_scaled
# probabilityOfQuitForTestData_Th_80[probabilityOfQuitForTestData>Threshold]=1
probabilityOfQuitForTestData_Th_80[probabilityOfQuitForTestData<=Threshold]=2
#print(probabilityOfQuitForTestData_Th_80)
print(confusion_matrix(test_target, probabilityOfQuitForTestData_Th_80))


[[ 436 1232]
 [  21 1466]]
[[135 404]
 [  4 509]]


- 임계값이 = 0.9 (임계값을 custom으로 설정하는 경우) 일때의 매트릭 (Threshold 설정)

    - 임계값 Threshold = 0.9 일때의 Train_scaled 데이터 대상 / Test_sacled 데이터 대상

In [124]:
Threshold = 0.9

# train_scaled
# probabilityOfQuitForTrainData_Th_90[probabilityOfQuitForTrainData>Threshold]=1
probabilityOfQuitForTrainData_Th_90[probabilityOfQuitForTrainData<=Threshold]=2
#print(probabilityOfQuitForTrainData_Th_90)
print(confusion_matrix(train_target, probabilityOfQuitForTrainData_Th_90))

# test_scaled
# probabilityOfQuitForTestData_Th_90[probabilityOfQuitForTestData>Threshold]=1
probabilityOfQuitForTestData_Th_90[probabilityOfQuitForTestData<=Threshold]=2
#print(probabilityOfQuitForTestData_Th_90)
print(confusion_matrix(test_target, probabilityOfQuitForTestData_Th_90))


[[ 312 1356]
 [  10 1477]]
[[ 99 440]
 [  2 511]]


## 여기까지의 결과 정리해서 논문 제출 필요 (Github Upload + 데이터 포함)